В этом примере мы покажем, как с помощью API Яндекс.Геокодера можно преобразовать адреса в геокоординаты, с которым можно работать в [Datalens](https://datalens.yandex.ru)

## Установка библиотек

 * requests для API Геокодера
 * clickhouse-driver для Clickhouse

In [1]:
%pip install requests

Note: you may need to restart the kernel to use updated packages.


## Получение ключа для API Геокодера

В переменную API_KEY нужно записать свой ключ для API Геокодера.
Получить ключ можно по [ссылке](https://yandex.ru/dev/maps/geocoder/)

In [10]:
API_KEY = ''

## Работа с API Геокодера

Напишем небольшой класс для работы с Геокодером

In [1]:
import requests
from dataclasses import dataclass

@dataclass
class YandexGeocoder:
    api_key: str
    geocoder_url: str = 'https://geocode-maps.yandex.ru/1.x'

    def adress_to_geopoint(self, address: str) -> str:
        """
        Преобразование адреса в геокоординаты в формате Datalens
        """
        response = requests.get(self.geocoder_url, params={
            'apikey': self.api_key,
            'geocode': address,
            'format': 'json',
        })
        response.raise_for_status()

        result = response.json()['response']['GeoObjectCollection']['featureMember']
        if not result:
            return None

        lat, lon = result[0]['GeoObject']['Point']['pos'].split(' ')
        return self._to_datalens_format(lon, lat)
    
    def _to_datalens_format(self, lon, lat):
        return f'[{lon},{lat}]'

Затем выгрузим данные из таблицы в переменную ch_data

In [8]:
import pandas as pd
df = pd.read_csv('Cuisine_rating.csv')
df = df[['Location']]
set_locations = list(set(df['Location']))
set_locations

['Upper West Side,NY',
 'Central Park,ny',
 'Riverdale,NY',
 'Market City, MY',
 'Upper East Side,NY',
 'Market City, NY',
 'Cedar Hill, NY',
 'Central Park,NY',
 'China Town, NY',
 'St. George,NY']

## Геокодирование

Преобразуем адреса магазинов из колонки ShopAddress в геокоординаты

In [11]:
geocoder = YandexGeocoder(api_key=API_KEY)

In [12]:
encoded_data = []
for adress in set_locations:
    s = geocoder.adress_to_geopoint(adress)
    print(adress, s)
    encoded_data.append(s)


Upper West Side,NY [40.786843,-73.975698]
Central Park,ny [40.782507,-73.965502]
Riverdale,NY [40.9043,-73.914388]
Market City, MY [42.90045,-103.088524]
Upper East Side,NY [40.774033,-73.955737]
Market City, NY [40.714627,-74.002863]
Cedar Hill, NY [42.544201,-73.77399]
Central Park,NY [40.782507,-73.965502]
China Town, NY [40.714627,-74.002863]
St. George,NY [40.644158,-74.073461]


In [13]:
len(encoded_data)

10

In [15]:
encoded_data

['[40.786843,-73.975698]',
 '[40.782507,-73.965502]',
 '[40.9043,-73.914388]',
 '[42.90045,-103.088524]',
 '[40.774033,-73.955737]',
 '[40.714627,-74.002863]',
 '[42.544201,-73.77399]',
 '[40.782507,-73.965502]',
 '[40.714627,-74.002863]',
 '[40.644158,-74.073461]']

In [14]:
import pandas as pd

In [18]:
df = pd.DataFrame(encoded_data, set_locations)

In [19]:
df

,0
"Upper West Side,NY","[40.786843,-73.975698]"
"Central Park,ny","[40.782507,-73.965502]"
"Riverdale,NY","[40.9043,-73.914388]"
"Market City, MY","[42.90045,-103.088524]"
"Upper East Side,NY","[40.774033,-73.955737]"
"Market City, NY","[40.714627,-74.002863]"
"Cedar Hill, NY","[42.544201,-73.77399]"
"Central Park,NY","[40.782507,-73.965502]"
"China Town, NY","[40.714627,-74.002863]"
"St. George,NY","[40.644158,-74.073461]"


In [20]:
df.to_csv("coord_city5.csv")